# 题目

> 给定一个字符串 (s) 和一个字符模式 (p) ，实现一个支持 '?' 和 '\*' 的通配符匹配。  
'?' 可以匹配任何单个字符。  
'\*' 可以匹配任意字符串（包括空字符串）。  
两个字符串完全匹配才算匹配成功。  
**说明:**  
s 可能为空，且只包含从 a-z 的小写字母。  
p 可能为空，且只包含从 a-z 的小写字母，以及字符 ? 和 \*。

# 方法一：动态规划

> 在模式 p 中，只会出现三种字符：小写字母、'?'和'\*'。其中只有'\*'的匹配是不确定的。  
可以用 $dp[i][j]$ 表示当前状态，即字符串 s 的前 $i$ 个字符和模式 p 的前 $j$ 个字符是否匹配。  
在状态转移时，考虑模式 p 的第 $j$ 个字符 $p_j$ 和字符串 s 的第 $i$ 个字符 $s_i$ 是否匹配：  
1. 若 $p_j$ 是小写字母，则 $s_i$ 必须与之相同，即有 $dp[i][j]=\left(s_i\right.$ 与 $p_j$ 相同 $) \wedge dp[i-1][j-1]$ （其中 '$\wedge$' 左右两边同时为真时表达式为真）；  
2. 若 $p_j$ 是'?'，则对 $s_i$ 没有任何要求，即有 $dp[i][j]=dp[i-1][j-1]$ ；
3. 若 $p_j$ 是'\*'，则同样对 $s_i$ 没有任何要求，状态转移方程分为使用'\*'和不使用'\*'两种情况 $dp[i][j]=dp[i][j-1] \vee dp[i-1][j]$ （其中 '$\wedge$' 左边表示不使用'\*'，右边表示使用'\*'）。  

> 归纳前两种情况，可得：$$d p[i][j]= \begin{cases}d p[i-1][j-1], & s_i \text { 与 } p_j \text { 相同或者 } p_j \text { 是问号 } \\ d p[i][j-1] \vee d p[i-1][j], & p_j \text { 是星号 } \\ \text { False, } & \text { 其它情况 }\end{cases}$$  

> 边界条件为：  
1. $dp[0][0]=True$ ，即字符串 s 和模式 p 均为空时，匹配成功；  
2. $dp[i][0]=False$ ，即空模式无法匹配非空字符串；
3. 只有当模式 p 的前 j 个字符均为'\*'时， $dp[0][j]=True$ 。

> 最终答案为：$dp[m][n]$ 。其中 $m,n$ 分别是字符串 s 和模式 p 的长度。

# 方法二：贪心

> 方法一的问题在于要记录动态规划的所有情况，导致较大的空间复杂度。  
对于模式 p 来说，连续的多个'\*'和单独的一个'\*'是等价的，因此模式 p 可以写成以下三种形式：  
1. $* u_1 * u_2 * u_3 * \cdots * u_x *$ 。只需要在字符串中先找到子串 $u_1$ ，再依次找到 $u_2,u_3 \cdots u_x$ 就可以匹配；  
2. $u_1 * u_2 * u_3 * \cdots * u_x *$ ，即开头没有'\*'。可以不断从开头对模式 p 和字符串 s 进行匹配，直到转化为第一种形式、匹配成功或匹配失败。
3. $* u_1 * u_2 * u_3 * \cdots * u_x$ ，即结尾没有'\*'。可以不断从结尾对模式 p 和字符串 s 进行匹配，直到转化为第一种形式、匹配成功或匹配失败。

> 在匹配过程中，模式 p 去掉匹配成功部分的子串不断在三种情况中转换。

## 复杂度

- 时间复杂度: 渐进复杂度为 $O(mn)$ ，其中 $m,n$ 分别为字符串 s 和模式 p 的长度。

> 字符串 s 和模式 p 中的每一个字符彼此间至多会进行一次匹配判断。平均复杂度为 $O(mlogn)$ 。

- 空间复杂度: $O(1)$ 。

> 只使用常数空间。

## 代码

In [1]:
def isMatch(s, p):
    def allStars(st, left, right):
        return all(st[i] == '*' for i in range(left, right))
    
    # 判断两个字符是否匹配
    def charMatch(u, v):
        return u == v or v == '?'

    sRight, pRight = len(s), len(p)  # 从最右边开始匹配
    
    # 情况1：模式p的末端字符不是*
    # 从右向左依次匹配每个字符，若模式p的字符不是*，则直接判断字符是否相互匹配
    while sRight > 0 and pRight > 0 and p[pRight - 1] != '*':
        if charMatch(s[sRight - 1], p[pRight - 1]):
            sRight -= 1
            pRight -= 1
        else:
            return False
    # 只有当两个字符同时走到头才算匹配成功
    if pRight == 0:
        return sRight == 0
    
    # 只要遍历到模式p中的*，执行以下代码
    # 开始从左往右进行匹配
    sIndex, pIndex = 0, 0  # 从左往右遍历字符串s和模式p，记录当前匹配字符的位置
    sRecord, pRecord = -1, -1  # pRecord记录了模式p上一个*的位置，sRecord则记录了字符串与*匹配的一个或一串字符的结尾位置（用于重新匹配）
    while sIndex < sRight and pIndex < pRight:
        
        # 情况2：此时的模式p开头与结尾都是*
        # 若当前模式字符是*，则向右继续查看模式p的下一个字符
        if p[pIndex] == '*':
            pIndex += 1
            # 每次模式p出现*，都要记录*在模式p的位置和其匹配的字符串s中的结尾字符位置
            sRecord, pRecord = sIndex, pIndex
        
        # 情况3：此时的模式p结尾是*，但开头不是*
        # 若当前模式字符不是*，则查看字符串s和模式p的当前字符能否匹配，若能，则向右继续匹配下一个字符
        elif charMatch(s[sIndex], p[pIndex]):
            sIndex += 1
            pIndex += 1
        # 若当前模式字符不是*，且字符串s和模式p的当前字符也不匹配，则查看是否有重新匹配的机会
        # 若sRecord为-1，说明模式p的开头字符不是*，不能重新匹配
        # 若sRecord+1=sRight，说明字符串s的下一个字符已经和模式p的尾部字符串匹配成功，若让当前的字符去匹配模式p前面的*，则模式p必然会有多余的字符无法匹配
        elif sRecord != -1 and sRecord + 1 < sRight:
            sRecord += 1  # 若重新匹配，则与*匹配的字符串s中的子串结尾位置向后挪动一位
            sIndex, pIndex = sRecord, pRecord  # 重新回到上一个*位置再进行匹配
        else:
            return False
    
    #最后，若字符串s中的所有字符已经匹配成功，但模式p中还有字符，则判断其剩余字符是否全为*，若是，返回True
    return allStars(p, pIndex, pRight)

#### 测试一

In [2]:
s = "aa"
p = "a"
isMatch(s, p)

False

#### 测试二

In [3]:
s = "adceb"
p = "*a*b"
isMatch(s, p)

True

#### 测试三

In [4]:
s = "acdcb"
p = "a*c?b"
isMatch(s, p)

False